In [1]:
import sys
sys.path.append('../scripts/')
from mcl import *

In [2]:
class ResetMcl(Mcl): 
    def __init__(self, envmap, init_pose, num, motion_noise_stds={"nn":0.19, "no":0.001, "on":0.13, "oo":0.2}, \
                 distance_dev_rate=0.14, direction_dev=0.05):
        super().__init__(envmap, init_pose, num, motion_noise_stds, distance_dev_rate, direction_dev)
        self.alphas = {}  #α値の記録（ランドマークの観測数ごと）
        
    def observation_update(self, observation):
        for p in self.particles:
            p.observation_update(observation, self.map, self.distance_dev_rate, self.direction_dev) 
        
        #alpha値の記録
        alpha = sum([p.weight for p in self.particles])
        obsnum = len(observation)
        if not obsnum in self.alphas: self.alphas[obsnum] = []
        self.alphas[obsnum].append(alpha)
            
        self.set_ml()
        self.resampling() #ここで重みの合計は1になる

In [ ]:
time_interval = 0.1
world = World(300, time_interval, debug=True) 

### 地図を生成して3つランドマークを追加 ###
m = Map()                                  
m.append_landmark(Landmark(-4,2))
m.append_landmark(Landmark(2,-3))
m.append_landmark(Landmark(3,3))
world.append(m)          

### ロボットを作る ###
initial_pose = np.array([0, 0, 0]).T
pf = ResetMcl(m, initial_pose, 100)
circling = EstimationAgent(time_interval, 0.2, 10.0/180*math.pi, pf)
r = Robot(initial_pose, sensor=Camera(m), agent=circling, color="red")
world.append(r)

world.draw()
